# Trial1: 그냥 버트 써서 classifier 통과시키기.

In [1]:
import json
import pandas as pd
# with open("data/fromgithub2.json","r") as f:
#     df = pd.read_json(f)
#     # posts = json.loads(f.read())

# df = df[["title","time","url"]]
# df["time"] = df["time"].astype("datetime64[s]",)
# df["visited"] = 0
# # df.to_csv("data/fromgithub2.csv")

import json
import pandas as pd

FILE = "data/database1.csv"
df = pd.read_csv(FILE)
df = df[["title","visited"]]


df

,title,visited
0,Jraft – the best blogging platform ever,1
1,"California, Love It and Leave It",0
2,"SpaceX Starlink has some hiccups as expected, ...",1
3,SpaceX Crew-1 [video],1
4,The Treehouse Is De-Plaformed,0
...,...,...
1487,U.S. Feds Seized Nearly $1B in Bitcoin from Wa...,0
1488,Facebook was used as a proxy by web scraping bots,1
1489,How is Visual Basic still ranked #6 programmin...,0
1490,Confessions of a voter fraud: I was a master a...,0


In [2]:
from datetime import datetime

# # features_df = df[["title","time","kids"]]
# # titles = df.loc[df["title"].isna()==False,"title"]
# df.dropna(subset=["title"])
# titles = df["title"]

# # features_df.loc[:,"kids"] = features_df["kids"].apply(lambda x: len(x))
# # features_df["time"] = features_df["time"].astype('datetime64[s]')
# titles.head()

In [3]:
# # fill dummy data
# y = [1 if i%2==0 else 0 for i in range(len(features))]
# y = np.array(y)

from sklearn.model_selection import train_test_split
RANDOM_STATE = 42

X_train, X_test, y_train, y_test = train_test_split(
    df["title"], df["visited"], test_size=0.33, random_state=RANDOM_STATE)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((999,), (493,), (999,), (493,))

In [4]:
import numpy as np
from transformers import DistilBertTokenizer, DistilBertModel 
from transformers import BertTokenizer, BertModel

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertModel.from_pretrained("distilbert-base-uncased")
# tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
# model = BertModel.from_pretrained("bert-base-uncased")


# tokenized = df["title"].apply(
#     (lambda x: tokenizer.encode(x, add_special_tokens=True)))
tokenized_train = X_train.apply(
    (lambda x: tokenizer.encode(x, add_special_tokens=True)))
tokenized_test = X_test.apply(
    (lambda x: tokenizer.encode(x, add_special_tokens=True)))

tmp = max([len(row) for row in tokenized_train])
max_len = max([len(row) for row in tokenized_test])
max_len = max(tmp, max_len)


padded_train = np.array([i + [0]*(max_len-len(i)) for i in tokenized_train.values])
padded_test = np.array([i + [0]*(max_len-len(i)) for i in tokenized_test.values])
attention_mask_train = np.where(padded_train!=0 , 1, 0)
attention_mask_test = np.where(padded_test!=0 , 1, 0)

/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
import torch

output_train = model(
    torch.tensor(padded_train),
    attention_mask=torch.tensor(attention_mask_train))
# output_test = model(torch.tensor(padded_test), attention_mask=attention_mask_test)

X_train = output_train[0][:,0,:].detach().numpy()
X_train.shape

(999, 768)

In [6]:
output_test = model(
    torch.tensor(padded_test),
    attention_mask=torch.tensor(attention_mask_test))

X_test = output_test[0][:,0,:].detach().numpy()
X_test.shape

(493, 768)

In [7]:
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import cross_val_score

print("DUMMY")
clf = DummyClassifier(random_state=RANDOM_STATE)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)
# cross_val_score(clf, X_test, y_test)

DUMMY


0.8275862068965517

In [8]:
from sklearn.linear_model import LinearRegression, LogisticRegression, RidgeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score
# from sklearn.metrics import roc_curve, roc_auc_score, RocCurveDisplay
from sklearn.metrics import roc_auc_score

print("LINEAR, LOGISTIC, RIDGE")
# clf = LinearRegression()
# clf = RidgeClassifier()
clfs = [
    LinearRegression(),
    # LogisticRegression(max_iter=2, n_jobs=-1,random_state=RANDOM_STATE),
    # LogisticRegression(),
    LogisticRegression(max_iter=5, solver="sag", n_jobs=-1,random_state=RANDOM_STATE),
    RidgeClassifier(),
    GaussianNB(),
]


for clf in clfs:
    print()
    clf.fit(X_train, y_train)
    print("acc: ", clf.score(X_test, y_test))
    print("crossval: ", cross_val_score(clf, X_test, y_test))
    # y_score = clf.pre(X_test)[:,1]
    y_score = clf.predict(X_test)
    print("auroc: ", roc_auc_score(y_test,y_score))
    # fpr, tpr = roc_curve(y_test, y_score)
    # display(RocCurveDisplay(fpr, tpr, ))


LINEAR, LOGISTIC, RIDGE

acc:  -4.677226181714537
crossval:  [-1.94293948e+00 -1.18443346e+06 -2.11428898e+08 -1.72837814e+01
 -2.93420149e+00]
auroc:  0.5695357554786621

acc:  0.8316430020283976


/home/ubuntu/.local/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/ubuntu/.local/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/ubuntu/.local/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/ubuntu/.local/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/ubuntu/.local/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/ubuntu/.local/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:35

crossval:  [0.83838384 0.82828283 0.82828283 0.82653061 0.82653061]
auroc:  0.5117647058823529

acc:  0.7991886409736308
crossval:  [0.76767677 0.77777778 0.77777778 0.78571429 0.74489796]
auroc:  0.5247549019607843

acc:  0.6267748478701826
crossval:  [0.63636364 0.67676768 0.56565657 0.57142857 0.59183673]
auroc:  0.592892156862745


# Trial2: fine tuing with torch.

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split

FILE = "data/database1.csv"
RANDOM_STATE = 42

df = pd.read_csv(FILE)

X_train, X_test, y_train, y_test = train_test_split(
    df["title"], df["visited"], test_size=0.33, random_state=RANDOM_STATE)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((999,), (493,), (999,), (493,))

In [11]:
import numpy as np
from transformers import DistilBertTokenizer, DistilBertModel 
from transformers import BertTokenizer, BertModel

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertModel.from_pretrained("distilbert-base-uncased")

def preprocess_text(X_text):
    tokenized = X_text.apply(
        (lambda x: tokenizer.encode(x, add_special_tokens=True)))
    
    max_len = max([len(row) for row in tokenized])
    print(max_len)
    # return text


preprocess_text(X_train)
preprocess_text(X_test)


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


31
27
